# RAG Quest

#### Environment variables + Libraries

In [1]:
import telebot
from time import sleep

import PyPDF2
# from docx import Document
from docx import Document

from dotenv import load_dotenv
import os

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings import GigaChatEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import DocArrayInMemorySearch

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [2]:
load_dotenv()

scope = os.getenv('SCOPE')
auth_data = os.getenv('AUTH_DATA')
bot_token = os.getenv('BOT_TOKEN')

Document loaders: 
[.pdf](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)
[.docx](https://python.langchain.com/docs/integrations/document_loaders/microsoft_word)

In [3]:
# # .pdf to .txt
# def convert_pdf_to_text(path):
#     pdf_file_obj = open(path, 'rb')
#     pdf_reader = PyPDF2.PdfReader(pdf_file_obj)
#     text = ''
#     for page_num in range(len(pdf_reader.pages)):
#         page_obj = pdf_reader.pages[page_num]
#         page_text = page_obj.extract_text()
#         page_text = page_text.replace('\n', ' ')
#         text += page_text
#     pdf_file_obj.close()
#     return text

# # .doc to .txt
# def convert_docx_to_text(path):
#     doc = Document(path)
#     text = ' '.join([paragraph.text for paragraph in doc.paragraphs])
#     return text

# paths = [
#     '/Users/edwardgrey/Desktop/RAGQuest/Положение о закупках товаров, работ, услуг_.docx.pdf',
#     '/Users/edwardgrey/Desktop/RAGQuest/Проведение закупки у единственного поставщика (подрядчика, исполнителя) (версия 3.0).docx',
#     '/Users/edwardgrey/Desktop/RAGQuest/Формирование Годового плана закупок (версия 3.0).docx',
# ]

# text = ''
# for path in paths:
#     if path.endswith('.pdf'):
#         text += convert_pdf_to_text(path)
#     elif path.endswith('.docx'):
#         text += convert_docx_to_text(path)

# with open('text.txt', 'w', encoding='utf-8') as f:
#     f.write(text)

In [4]:
loader = TextLoader("text_manual.txt")
text_documents = loader.load()
text_documents

[Document(page_content='1. ОБЩИЕ ПОЛОЖЕНИЯ\n1.1. Предмет и цели регулирования\n1.1.1. Настоящее Положение о закупках товаров, работ, услуг ПАО «Компания 1»\n(далее - Положение) разработано в целях своевременного и полного обеспечения\nпотребностей ПАО «Компания 1» (далее также - Общество) и обществ, определенных в\nпункте 1.2.35 настоящего Положения (далее также – Общества Компания 1), в товарах, работах, услугах, совершенствования порядка и повышения эффективности закупок.\n\n1.1.2. Положение разработано в соответствии с Конституцией Российской\nФедерации, Гражданским кодексом Российской Федерации, другими федеральными\nзаконами и иными нормативными правовыми актами Российской Федерации,\nПоложением о закупках товаров, работ, услуг ПАО «Компания 2» и Компаний Группы\nКомпания 2, принятыми в соответствии с указанными документами локальными\nнормативными актами ПАО «Компания 1», общепринятыми правилами, сложившимися в\nмировой практике в сфере закупок.\n\n1.1.3. Настоящее Положение регу

#### Setting up LLM model


[GigaChat](https://developers.sber.ru/docs/ru/gigachat/overview)

In [5]:
gigachat_model = GigaChat(credentials=auth_data, verify_ssl_certs=False)

In [6]:
gigachat_model.get_num_tokens("Сколько токенов в этой строке")

6

In [7]:
gigachat_model.invoke("2+2=")

AIMessage(content='<fuse>calculator(query="2+2")</fuse>', response_metadata={'token_usage': Usage(prompt_tokens=15, completion_tokens=18, total_tokens=33), 'model_name': 'GigaChat:3.1.24.3', 'finish_reason': 'stop'})

In [8]:
parser = StrOutputParser()

chain = gigachat_model | parser
chain.invoke("Скощько спутников у земли?")

'Точное количество спутников на орбите Земли может меняться со временем, так как спутники запускаются и выводятся из эксплуатации. По данным на 2023 год, на низкой околоземной орбите (от 160 до 2000 километров над поверхностью Земли) находится около 4500 активных спутников. Это включает в себя спутники связи, навигации, метеорологии, дистанционного зондирования Земли и другие.\n\nОднако стоит отметить, что это число не включает в себя космический мусор - отработавшие спутники, обломки ракет-носителей и другие объекты, которые остаются на орбите после завершения своей миссии. По оценкам, общее количество космического мусора на низкой околоземной орбите превышает 5000 единиц.'

#### Prompt templates

In [9]:
template = """
Ваша задача состоит в том, чтобы анализировать представленный запрос и предоставлять ответы, опираясь на конкретные документы, правила и требования. Важно не только давать общие разъяснения, но и приводить точные ссылки на пункты документов, которые подтверждают ваш ответ. Это поможет гарантировать, что ответы являются не только точными и релевантными, но и полностью обоснованными.
При ответе на вопросы, особенно те, которые касаются правил, норм и требований, активно используйте номера пунктов, статей и других разделов релевантных документов. Если информация, необходимая для ответа, отсутствует в предоставленном контексте, и вы не можете найти конкретные пункты в документах, используйте формулировку 'Информация не найдена'.
Обратите внимание, что ваш ответ должен быть максимально точным и детализированным, идентифицируя не только релевантные разделы документов, но и объясняя, как они применимы к конкретному запросу.

Вопрос:
{question}

Контекст:
{context}

Пожалуйста, предоставьте ваш ответ ниже, не забывая ссылаться на конкретные документы и пункты.
"""

prompt = ChatPromptTemplate.from_template(template)

In [10]:
chain = prompt | gigachat_model | parser

chain.invoke({
    "context": "У Саши красный велосипед",
    "question": "Какого цвета велосиепед Саши?"
})

'Сашин велосипед красного цвета. Информация подтверждается контекстом, который указывает на то, что у Саши красный велосипед.'

In [11]:
def extract_question(model, parser, text):
    chain = model | parser

    prompt_for_extraction = f"Ваша задача - анализировать представленный текст и идентифицировать в нём все ключевые вопросы, которые требуют ответа. Следует обратить особое внимание на предложения, оканчивающиеся вопросительным знаком, и контекст, который может указывать на неявные запросы за разъяснениями или дополнительной информацией. Важно не только выделить эти вопросы, но и предоставить краткую формулировку каждого из них, сохраняя при этом точность и суть запроса. После идентификации вопросов составьте их перечень в чёткой и сжатой форме, так чтобы они могли быть поняты без контекста первоначального обращения. Текст: \"{text}\""
    model_response = chain.invoke(prompt_for_extraction)
    
    return model_response

In [12]:
test = "Уважаемы коллеги, добрый день!Если в процессе инициации закупки возникла необходимость в уменьшении НМЦ закупки, то нужно ли делать корректировку в ГПЗ?Например, в процессе согласования договора с ВЗЛ, у заказчика немного уменьшилась потребность в чел./часах на выполнение определённых работ, следовательно, снизилась стоимость договора. Состав работ и команда специалистовпри этом не изменились. Нужна ли корректировка в этом случае?"
extract_question(gigachat_model, parser, test)

'1. Нужно ли делать корректировку в ГПЗ при уменьшении НМЦ закупки?\n2. Требуется ли корректировка в ГПЗ, если потребность в чел./часах на выполнение работ снизилась, но состав работ и команда специалистов остались неизменными?'

#### Splitting the text

[Text Splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/)

![Example Image](embedding.png)

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

[Chunks](https://chunkviz.up.railway.app)

#### Loading text into the vector store

In [14]:
embeddings = GigaChatEmbeddings(credentials=auth_data, verify_ssl_certs=False)

[Sentence Transformers](https://www.sbert.net)

In [15]:
vectorstore = DocArrayInMemorySearch.from_documents(documents, embeddings)

/home/viktor/Projects/Univer/DigitalDepartment/2sem/Hackathon/rag_quest/.venv/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


#### RAG

![RAG](rag.png)

In [16]:
retriever = vectorstore.as_retriever()
setup = RunnableParallel(context=retriever, question=RunnablePassthrough())

In [17]:
chain = (setup | prompt | gigachat_model | parser)

In [18]:
chain.invoke("Сколько спутников у Сатурна?")

'Из предоставленных документов не удалось найти информацию о количестве спутников Сатурна.'

In [19]:
query = chain.invoke("Если в процессе инициации закупки возникла необходимость в уменьшении НМЦ закупки, то нужно ли делать корректировку в ГПЗ? Например, в процессе согласования договора с ВЗЛ, у заказчика немного уменьшилась потребность в чел./часах на выполнение определённых работ, следовательно, снизилась стоимость договора. Состав работ и команда специалистов при этом не изменились. Нужна ли корректировка в этом случае?")
relevant_documents = retriever.get_relevant_documents(query)
relevant_documents

[Document(page_content='изменение формы закупки; изменение Организатора / Координатора закупки; разделение на лоты / процедуры, объединение в один лот / процедуру; изменение кода услуги; изменение категории закупки; В случае уточнения предмета закупки / предмета договора при инициировании закупки корректировка записей ГПЗ не требуется. Ежемесячно Заказчик проверяет факт инициации запланированных в утвержденном ГПЗ закупок. Не инициированные в рамках месяца или ранее (в пределах года планирования) закупки подлежат переносу на иной срок (в пределах года) либо исключению из ГПЗ не позднее 2 р.д. с начала месяца, следующего за месяцем, в котором планировалось инициировать закупку.  Ответственность за актуальность сведений в ГПЗ несет Заказчик. Не инициированные в пределах месяца закупки, по которым Заказчиками в соответствии с п. 5.3.6 настоящего стандарта не были оформлены корректировки в ГПЗ, могут быть в одностороннем порядке исключены из ГПЗ либо перенесены на иной срок (в пределах год

[Reference](https://docs.google.com/spreadsheets/d/1Gc5N8DiMcj1gmisO9Ij8L_8HaiKPBHVR/edit#gid=1142278491)

#### Telegram Bot

In [20]:
user_conversations = {}
bot = telebot.TeleBot(bot_token)

In [21]:
chain = ( setup | prompt| gigachat_model | parser )

@bot.message_handler(content_types=['audio', 'video', 'document', 'photo', 'sticker', 'voice', 'location', 'contact'])
def not_text(message):
    user_id = message.chat.id
    bot.send_message(user_id, 'Я работаю только с текстовыми сообщениями!')

@bot.message_handler(content_types=['text'])
def handle_text_message(message):
    user_id = message.chat.id
    extracted_question = extract_question(gigachat_model, parser, message.text)

    response = chain.invoke(extracted_question)

    bot.send_message(user_id, response)
    sleep(2)

In [22]:
# bot.polling(none_stop=True)

#### Count cross-encoder score

In [23]:
from FlagEmbedding import BGEM3FlagModel
cross_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False)

def cross_score(model_answer, ans):
    score =  cross_model.compute_score([ans, model_answer], max_passage_length=128, weights_for_different_modes=[1.0, 1.0, 1.0])['colbert']
    return score

/home/viktor/Projects/Univer/DigitalDepartment/2sem/Hackathon/rag_quest/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Fetching 22 files: 100%|██████████| 22/22 [00:00<00:00, 49292.03it/s]


In [24]:
sent1 = "Я пошел в магазин"
sent2 = "В магазине взял хлеб"
score = cross_score(sent1, sent2)
print(score)

0.7973787188529968


#### Import Yandex model

In [25]:
from langchain_community.chat_models import ChatYandexGPT
from langchain_core.messages import HumanMessage, SystemMessage
import grpc

In [ ]:
!curl -sSL https://storage.yandexcloud.net/yandexcloud-yc/install.sh | bash

##### Yandex authentication

In [71]:
!yc config profile create testprofile

ERROR: profile 'testprofile' already exists


In [72]:
!source .env

In [73]:
!yc config set token $YANDEX_OATH_TOKEN

In [ ]:
!mkdir keys
!yc iam key create --service-account-name testprofile --output keys/yandex_auth_key_testprofile.json --folder-id $YC_FOLDER_ID

In [76]:
!yc config profile create testprofile
!yc config set service-account-key keys/yandex_auth_key_testprofile.json

ERROR: profile 'testprofile' already exists


Update IAM-token in .env

In [77]:
!sed -i "s/YC_IAM_TOKEN=.*$/YC_IAM_TOKEN=$(yc iam create-token)/g" .env


In [60]:
from dotenv import load_dotenv
load_dotenv(override=True)

YC_IAM_TOKEN = os.getenv('YC_IAM_TOKEN')
YC_API_KEY = os.getenv('YC_API_KEY')
YC_FOLDER_ID = os.getenv('YC_FOLDER_ID')

In [63]:
yandexgpt_model = None
if YC_IAM_TOKEN is not None:
    yandexgpt_model = ChatYandexGPT()

In [65]:
# Check is model works
if yandexgpt_model is not None:
    print(yandexgpt_model.invoke("Сколько континентов на Земле?"))
    print(yandexgpt_model.get_num_tokens("Сколько токенов в этой строке"))

content='Обычно на планете Земля выделяют шесть континентов:\n\n1. **Африка**.\n2. **Евразия**, которая состоит из меньших континентов — Европы и Азии.\n3. **Северная Америка**.\n4. **Южная Америка**.\n5. **Антарктида**.\n\nОднако некоторые географы могут рассматривать дополнительные континенты или включать в этот список другие территории. Также можно встретить разделение суши на Старый Свет (в составе Европа + Азия) и Новый Свет (в составе Северная Америка + Южная Америка). \n\nВажно отметить, что нет единого мнения о том, сколько континентов существует на Земле. Это вопрос географии и определения, и он может зависеть от различных подходов и критериев.'
31


#### Choose best template and model

In [66]:
question_and_context = \
"""
    Вопрос: {question}

    Контекст: {context}
"""
templates =[
    # Шаблон, который просит написать кратко и конкретно
    f"""
    {question_and_context}

    Ответь на вопрос, строго опираясь на контекст, кратко и конкретно, без лишней информации. При этом укажи ссылки на пункты документов, откуда ты взял информацию. Объясни свой ответ подробно в вежливом, официальном стиле. 
    Если информация, необходимая для ответа, отсутствует в предоставленном контексте, и вы не можете найти конкретные пункты в документах, используйте формулировку 'Информация не найдена.
    """,
    # Шаблон, который просит написать подробнее.
    f"""
    {question_and_context}
    Задача: дан вопрос и контекст. Важно ответить на вопрос только на осВ контекстенове знаний, полученных из контекста.  есть числа, обозначающие пункты в объемном документе, при ответе на вопрос следует указывать ссылки на пункты, на которые опирается ответ.
    Ответ должен быть написан с использованием современных правил русского языка. При этом стиль общения должен быть официальным, важно быть дружелюбным и доброжелательным.
    Если информация, необходимая для ответа, отсутствует в предоставленном контексте, и вы не можете найти конкретные пункты в документах, используйте формулировку 'Информация не найдена.
    """,
    # Шаблон, который просит написать подробно.
    f"""
    Ваша задача состоит в том, чтобы анализировать представленный запрос и предоставлять ответы, опираясь на конкретные документы, правила и требования. Важно не только давать общие разъяснения, но и приводить точные ссылки на пункты документов, которые подтверждают ваш ответ. Это поможет гарантировать, что ответы являются не только точными и релевантными, но и полностью обоснованными.
    При ответе на вопросы, особенно те, которые касаются правил, норм и требований, активно используйте номера пунктов, статей и других разделов релевантных документов. Если информация, необходимая для ответа, отсутствует в предоставленном контексте, и вы не можете найти конкретные пункты в документах, используйте формулировку 'Информация не найдена'.
    Обратите внимание, что ваш ответ должен быть максимально точным и детализированным, идентифицируя не только релевантные разделы документов, но и объясняя, как они применимы к конкретному запросу.

    {question_and_context}

    Пожалуйста, предоставьте ваш ответ ниже, не забывая ссылаться на конкретные документы и пункты.
    """,
    # Шаблон, который просит написать максимально подробно.
    f"""
    Ваша задача состоит в том, чтобы анализировать представленный запрос и предоставлять ответы, опираясь на конкретные документы, правила и требования. Важно не только давать общие разъяснения, но и приводить точные ссылки на пункты документов, которые подтверждают ваш ответ. Это поможет гарантировать, что ответы являются не только точными и релевантными, но и полностью обоснованными.
    Ответ должен быть написан с использованием современных правил русского языка. При этом стиль общения должен быть официальным, важно быть дружелюбным и доброжелательным.
    При ответе на вопросы, особенно те, которые касаются правил, норм и требований, активно используйте номера пунктов, статей и других разделов релевантных документов. Если информация, необходимая для ответа, отсутствует в предоставленном контексте, и вы не можете найти конкретные пункты в документах, используйте формулировку 'Информация не найдена'.
    Обратите внимание, что ваш ответ должен быть максимально точным и детализированным, идентифицируя не только релевантные разделы документов, но и объясняя, как они применимы к конкретному запросу.

    {question_and_context}

    Пожалуйста, предоставьте ваш ответ ниже, не забывая ссылаться на конкретные документы и пункты.
    """
]

In [67]:
class Test:
    def __init__(self, question, true_answer):
        self.question = question
        self.true_answer = true_answer

import pandas as pd
reference = pd.read_excel('reference.xlsx')
records = reference.to_dict('records')
letter_key = 'letter'
answer_key = 'answer '

tests = []
for record in records:
    tests.append(Test(record[letter_key], record[answer_key]))


tests = [
    Test('Являются ли решения Закупочной комиссии обязательными для Заказчика?', 
         """1.4.12.
            Решения Закупочной комиссии обязательны для Заказчика."""),
    Test('Кто осуществляет подготовку заседаний Закупочной комиссии?', 
         """1.4.13.
            Подготовку заседаний Закупочной комиссии осуществляет Организатор.""")
]

In [68]:
def get_cross_score_for_test(chain, test: Test):

    answer = chain.invoke(test.question)
    score = cross_score(answer, test.true_answer)
    return score

def count_mean_score_on_tests(chain, tests: list[Test]):    
    score_sum = 0
    for test in tests:
        score_sum += get_cross_score_for_test(chain=chain, test=test)
    return score_sum / len(tests)

In [69]:
models = [gigachat_model]
models.append(yandexgpt_model)

In [70]:
best_template_mean_score = None
best_template = None
best_model = None

for model in  models:
    for template in templates:
        prompt = ChatPromptTemplate.from_template(template)
        chain = (setup | prompt | model | parser)
        current_mean_score = count_mean_score_on_tests(chain, tests)
        
        if best_template_mean_score is None or current_mean_score > best_template_mean_score:
            best_template_mean_score = current_mean_score
            best_template = template
            best_model = model

print('best_template = ', best_template)
print('best_model = ', best_model)
print('best_template_mean_score = ', best_template_mean_score)

AUTHENTICATION ERROR
AUTHENTICATION ERROR


best_template =  
    Ваша задача состоит в том, чтобы анализировать представленный запрос и предоставлять ответы, опираясь на конкретные документы, правила и требования. Важно не только давать общие разъяснения, но и приводить точные ссылки на пункты документов, которые подтверждают ваш ответ. Это поможет гарантировать, что ответы являются не только точными и релевантными, но и полностью обоснованными.
    При ответе на вопросы, особенно те, которые касаются правил, норм и требований, активно используйте номера пунктов, статей и других разделов релевантных документов. Если информация, необходимая для ответа, отсутствует в предоставленном контексте, и вы не можете найти конкретные пункты в документах, используйте формулировку 'Информация не найдена'.
    Обратите внимание, что ваш ответ должен быть максимально точным и детализированным, идентифицируя не только релевантные разделы документов, но и объясняя, как они применимы к конкретному запросу.

    
    Вопрос: {question}

    Конте